In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pymongo
import datetime
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import sem

In [2]:
file_path1 = "Resources/COVID-19_Hospital_Data_Coverage_Detail.csv"
file_path2 = "Resources/us_state_vaccinations_cleansed.csv"

In [3]:
file_df1 = pd.read_csv(file_path1)
file_df2 = pd.read_csv(file_path2)

In [4]:
file_df1.head(5)

,state,ccn,facility_name,street_address,city,zip_code,fips_county,provider_subtype,reporting_source,dates_with_missing_data,...,on_hand_supply_therapeutic_b_bamlanivimab_courses,previous_week_therapeutic_b_bamlanivimab_courses_used,previous_week_personnel_covid_vaccinated_doses_administered,total_personnel_covid_vaccinated_doses_none,total_personnel_covid_vaccinated_doses_one,total_personnel_covid_vaccinated_doses_all,total_personnel,previous_week_patients_covid_vaccinated_doses_one,previous_week_patients_covid_vaccinated_doses_all,Georeference
0,AR,040007,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,119,Short Term,tt,[],...,True,True,True,True,True,True,True,True,True,NaN
1,AR,040161,ARKANSAS HEART HOSPITAL-ENCORE,1901 ENCORE WAY,BRYANT,72022.0,125,Short Term,tt,"['2021-12-01', '2021-12-02']",...,False,False,False,False,False,False,False,False,False,NaN
2,AR,042009,REGENCY HOSPITAL OF NORTHWEST ARKANSAS,609 WEST MAPLE AVENUE,SPRINGDALE,72764.0,143,Long Term,tt,[],...,True,True,False,False,False,False,False,False,False,POINT (-94.136464 36.181394)
3,CT,070016,SAINT MARY'S HOSPITAL,56 FRANKLIN STEET,WATERBURY,6706.0,9,Short Term,tt,[],...,True,True,False,False,False,False,False,False,False,NaN
4,IA,161342,MONROE COUNTY HOSPITAL,6580 165TH STREET,ALBIA,52531.0,135,Critical Access Hospitals,tt,[],...,False,False,False,False,False,False,False,False,False,POINT (-92.796674 41.044816)


In [6]:
file_df2.head(5)

,date,location,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,11/26/21,Alabama,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13342,2721.0,NaN
1,11/27/21,Alabama,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664,2175.0,NaN
2,11/28/21,Alabama,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10771,2197.0,NaN
3,11/29/21,Alabama,AL,5229021.0,7802940.0,2767489.0,46.14,106.65,2262464.0,56.44,159.14,NaN,10573,2156.0,0.670
4,11/30/21,Alabama,AL,5242493.0,7834540.0,2772024.0,46.18,106.92,2264466.0,56.54,159.78,13472.0,10309,2103.0,0.669


In [7]:
file_df1.dtypes

state                                                object
ccn                                                  object
facility_name                                        object
street_address                                       object
city                                                 object
                                                      ...  
total_personnel_covid_vaccinated_doses_all             bool
total_personnel                                        bool
previous_week_patients_covid_vaccinated_doses_one      bool
previous_week_patients_covid_vaccinated_doses_all      bool
Georeference                                         object
Length: 97, dtype: object

In [83]:
file_df1.count()

state                                                5955
ccn                                                  5955
facility_name                                        5955
street_address                                       5955
city                                                 5955
                                                     ... 
total_personnel_covid_vaccinated_doses_all           5955
total_personnel                                      5955
previous_week_patients_covid_vaccinated_doses_one    5955
previous_week_patients_covid_vaccinated_doses_all    5955
Georeference                                         5282
Length: 97, dtype: int64

In [8]:
print(file_df1.columns)

Index(['state', 'ccn', 'facility_name', 'street_address', 'city', 'zip_code',
       'fips_county', 'provider_subtype', 'reporting_source',
       'dates_with_missing_data', 'certified_bed_count', 'component_hospitals',
       'week_ending', 'days_at_100%', 'percentage_of_required_fields_reported',
       'hospital_required_to_report_wed_only', 'total_beds',
       'all_adult_hospital_beds', 'inpatient_beds',
       'all_adult_hospital_inpatient_beds', 'inpatient_beds_used',
       'all_adult_hospital_inpatient_bed_occupied', 'total_icu_beds',
       'total_staffed_adult_icu_beds', 'icu_beds_used',
       'staffed_adult_icu_bed_occupancy', 'total_ventilators',
       'ventilators_used',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
       'total_adult_patients_hospitalized_confirmed_covid',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
       'total_pediatric_patients_hospitalized_confirmed_covid',
       'inpatient_ventilator

In [9]:
df1 = file_df1[['state', 'facility_name', 'street_address', 'city', 'zip_code', 'week_ending', 'certified_bed_count', 'total_beds', 'all_adult_hospital_beds', 'inpatient_beds', 'all_adult_hospital_inpatient_beds', 'inpatient_beds_used', 'all_adult_hospital_inpatient_bed_occupied', 'total_icu_beds', 'icu_beds_used', 'total_adult_patients_hospitalized_confirmed_covid', 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid', 'total_pediatric_patients_hospitalized_confirmed_covid', 'staffed_icu_adult_patients_confirmed_and_suspected_covid', 'staffed_icu_adult_patients_confirmed_covid', 'deaths_covid']]

In [10]:
print(df1.head)

<bound method NDFrame.head of      state                           facility_name         street_address  \
0       AR                CHI-ST VINCENT INFIRMARY  TWO ST VINCENT CIRCLE   
1       AR          ARKANSAS HEART HOSPITAL-ENCORE        1901 ENCORE WAY   
2       AR  REGENCY HOSPITAL OF NORTHWEST ARKANSAS  609 WEST MAPLE AVENUE   
3       CT                   SAINT MARY'S HOSPITAL      56 FRANKLIN STEET   
4       IA                  MONROE COUNTY HOSPITAL      6580 165TH STREET   
...    ...                                     ...                    ...   
5950    LA       JENNINGS AMERICAN LEGION HOSPITAL        1634 ELTON ROAD   
5951    PA                  HERITAGE VALLEY BEAVER  1000 DUTCH RIDGE ROAD   
5952    WA        OVERLAKE HOSPITAL MEDICAL CENTER      1035-116TH AVE NE   
5953    FL              KINDRED HOSPITAL MELBOURNE        765 W NASA BLVD   
5954    NC  NOVANT HEALTH MINT HILL MEDICAL CENTER   8201 HEALTHCARE LOOP   

             city  zip_code             week_

In [11]:
df1.head()

,state,facility_name,street_address,city,zip_code,week_ending,certified_bed_count,total_beds,all_adult_hospital_beds,inpatient_beds,...,inpatient_beds_used,all_adult_hospital_inpatient_bed_occupied,total_icu_beds,icu_beds_used,total_adult_patients_hospitalized_confirmed_covid,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid,total_pediatric_patients_hospitalized_confirmed_covid,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,deaths_covid
0,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
1,AR,ARKANSAS HEART HOSPITAL-ENCORE,1901 ENCORE WAY,BRYANT,72022.0,12/02/2021 12:00:00 AM,25,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,AR,REGENCY HOSPITAL OF NORTHWEST ARKANSAS,609 WEST MAPLE AVENUE,SPRINGDALE,72764.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
3,CT,SAINT MARY'S HOSPITAL,56 FRANKLIN STEET,WATERBURY,6706.0,12/02/2021 12:00:00 AM,347,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,IA,MONROE COUNTY HOSPITAL,6580 165TH STREET,ALBIA,52531.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [12]:
df1.loc

In [13]:
df1.to_csv('Resources/COVID-19_Hospital_Data_Coverage_Detail_cleansed.csv', index = False)

In [14]:
df1

,state,facility_name,street_address,city,zip_code,week_ending,certified_bed_count,total_beds,all_adult_hospital_beds,inpatient_beds,...,inpatient_beds_used,all_adult_hospital_inpatient_bed_occupied,total_icu_beds,icu_beds_used,total_adult_patients_hospitalized_confirmed_covid,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid,total_pediatric_patients_hospitalized_confirmed_covid,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,deaths_covid
0,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
1,AR,ARKANSAS HEART HOSPITAL-ENCORE,1901 ENCORE WAY,BRYANT,72022.0,12/02/2021 12:00:00 AM,25,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,AR,REGENCY HOSPITAL OF NORTHWEST ARKANSAS,609 WEST MAPLE AVENUE,SPRINGDALE,72764.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
3,CT,SAINT MARY'S HOSPITAL,56 FRANKLIN STEET,WATERBURY,6706.0,12/02/2021 12:00:00 AM,347,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,IA,MONROE COUNTY HOSPITAL,6580 165TH STREET,ALBIA,52531.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5950,LA,JENNINGS AMERICAN LEGION HOSPITAL,1634 ELTON ROAD,JENNINGS,70546.0,12/02/2021 12:00:00 AM,49,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
5951,PA,HERITAGE VALLEY BEAVER,1000 DUTCH RIDGE ROAD,BEAVER,15009.0,12/02/2021 12:00:00 AM,282,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
5952,WA,OVERLAKE HOSPITAL MEDICAL CENTER,1035-116TH AVE NE,BELLEVUE,98004.0,12/02/2021 12:00:00 AM,218,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
5953,FL,KINDRED HOSPITAL MELBOURNE,765 W NASA BLVD,MELBOURNE,32901.0,12/02/2021 12:00:00 AM,60,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [15]:
print(df1['state'])

0       AR
1       AR
2       AR
3       CT
4       IA
        ..
5950    LA
5951    PA
5952    WA
5953    FL
5954    NC
Name: state, Length: 5955, dtype: object


In [18]:
df3 = pd.merge(df1, file_df2)
df3

,state,facility_name,street_address,city,zip_code,week_ending,certified_bed_count,total_beds,all_adult_hospital_beds,inpatient_beds,...,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7724,2559.0,NaN
1,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5353,1774.0,NaN
2,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6615,2192.0,NaN
3,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,1823691.0,49.30,116.69,1487634.0,60.43,160.35,NaN,7087,2348.0,0.728
4,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,1824522.0,49.32,116.78,1488357.0,60.46,160.95,2621.0,7058,2339.0,0.726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40343,VI,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,820.0,12/02/2021 12:00:00 AM,160,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199,1872.0,NaN
40344,VI,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,820.0,12/02/2021 12:00:00 AM,160,7.0,7.0,7.0,...,60068.0,48.92,112.31,52002.0,56.51,116.91,NaN,214,2013.0,0.961
40345,VI,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,820.0,12/02/2021 12:00:00 AM,160,7.0,7.0,7.0,...,60195.0,49.00,112.72,52087.0,56.63,117.19,434.0,222,2089.0,0.962
40346,VI,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,820.0,12/02/2021 12:00:00 AM,160,7.0,7.0,7.0,...,60503.0,49.10,113.37,52184.0,56.92,119.77,688.0,272,2559.0,0.947
